### Sample sentiment dataset

In [11]:
import numpy as np
from cvxopt import matrix, solvers

In [1]:
texts = [
    "movie was amazing and fun",
    "i loved the film",
    "what a fantastic experience",
    "movie was boring and slow",
    "i hated the film",
    "terrible and disappointing"
]

In [2]:
labels = [1, 1, 1, -1, -1, -1]

### Convert text → TF-IDF vectors

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),   # capture phrases like "not good"
    stop_words='english'
)

In [7]:
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)

### Normalize features

In [8]:
X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)

### RBF Kernel

In [9]:
def rbf_kernel(X, gamma=0.5):
    sq_dist = (
        np.sum(X**2, axis=1).reshape(-1,1)
        + np.sum(X**2, axis=1)
        - 2 * X @ X.T
    )
    return np.exp(-gamma * sq_dist)

In [10]:
K = rbf_kernel(X)

### Solve SVM dual using QP (Lagrangian)

In [12]:
n = X.shape[0]
C = 1.0

In [13]:
P = matrix(np.outer(y, y) * K, tc='d')
q = matrix(-np.ones(n), tc='d')

In [14]:
G = matrix(np.vstack([-np.eye(n), np.eye(n)]), tc='d')
h = matrix(np.hstack([np.zeros(n), np.ones(n) * C]), tc='d')

In [15]:
A = matrix(y.reshape(1, -1), tc='d')
b = matrix(0.0, tc='d')

In [16]:
solvers.options['show_progress'] = False
solution = solvers.qp(P, q, G, h, A, b)

In [17]:
alphas = np.array(solution['x']).flatten()

### Identify support vectors

In [19]:
tol = 1e-5
sv = alphas > tol

In [20]:
alphas_sv = alphas[sv]
X_sv = X[sv]
y_sv = y[sv]

### Compute bias term 𝑏

In [21]:
K_sv = K[sv][:, sv]

In [22]:
b = np.mean(y_sv - np.sum(alphas_sv * y_sv * K_sv, axis=1))

### Prediction function (non-linear SVM)

In [23]:
def predict(text):
    x = vectorizer.transform([text]).toarray()
    x = (x - X.mean(axis=0)) / (X.std(axis=0) + 1e-8)

    K_test = np.exp(
        -0.5 * (
            np.sum((X_sv - x)**2, axis=1)
        )
    )

    score = np.sum(alphas_sv * y_sv * K_test) + b
    return "Positive" if score >= 0 else "Negative"

### Test sentiment prediction

In [24]:
print(predict("movie was not good at all"))
print(predict("i really loved the movie"))

Negative
Positive
